In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import explode, split, regexp_replace
from pyspark.sql.types import StructType, StructField, StringType
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
import matplotlib.pyplot as plt

In [2]:
# create a SparkSession
spark = SparkSession.builder.appName("TwitterStreaming").getOrCreate()

# create a StreamingContext with batch interval of 1 second
ssc = StreamingContext(spark.sparkContext, 1)
sqlContext = SQLContext(spark)

# create a DStream that reads data from port 5554
lines = ssc.socketTextStream("localhost", 5554)

def preprocessing(lines):
    words = lines.map(lambda x: x.strip()) \
                 .filter(lambda x: x != '') \
                 .toDF(schema=StringType()) \
                 .select(explode(split('value', '\s+')).alias('word')) \
                 .na.replace('', None) \
                 .na.drop() \
                 .withColumn('word', regexp_replace('word', r'http\S+', '')) \
                 .withColumn('word', regexp_replace('word', '@\w+', '')) \
                 .withColumn('word', regexp_replace('word', '#', '')) \
                 .withColumn('word', regexp_replace('word', 'RT', '')) \
                 .withColumn('word', regexp_replace('word', ':', ''))
    return words

# preprocess non-empty RDDs and store clean tweets in a Spark DataFrame
clean_tweets = lines.foreachRDD(lambda rdd: process_rdd(rdd) if not rdd.isEmpty() else None)

23/02/26 18:45:26 WARN Utils: Your hostname, Kaustubhs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.21 instead (on interface en0)
23/02/26 18:45:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/26 18:45:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
# start the streaming context
ssc.start()

23/02/26 18:45:53 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:45:53 WARN BlockManager: Block input-0-1677465953600 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:45:54 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:45:54 WARN BlockManager: Block input-0-1677465953800 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:45:54 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:45:54 WARN BlockManager: Block input-0-1677465954200 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:45:54 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:45:54 WARN BlockManager: Block input-0-1677465954600 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:45:55 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:45:55 WARN BlockManager: Block input-0-1677465955600 replicated to

In [4]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns

23/02/26 18:45:58 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:45:58 WARN BlockManager: Block input-0-1677465958600 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:45:59 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:45:59 WARN BlockManager: Block input-0-1677465958800 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:45:59 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:45:59 WARN BlockManager: Block input-0-1677465959600 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:46:00 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:46:00 WARN BlockManager: Block input-0-1677465959800 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:46:00 ERROR JobScheduler: Error running job streaming job 1677465954000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most

23/02/26 18:46:01 ERROR JobScheduler: Error running job streaming job 1677465957000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/var/folders/fc/dcskpc694gg68q4bvnmpm3mc0000gn/T/ipykernel_59394/72639139.py", line 27, in <lambda>
    spark.createDataFrame(preprocessing(rdd)) if not rdd.isEmpty() else None)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/sql/session.py", line 658, in createDataFrame
    raise TypeError("data is already a DataFrame")
TypeError: data is already a DataFrame

	at org.apache.spark.streaming.api.python.TransformFunction.callPythonTransformFunction(PythonDStream.scala:95)
	at org.ap

23/02/26 18:46:02 ERROR JobScheduler: Error running job streaming job 1677465961000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/var/folders/fc/dcskpc694gg68q4bvnmpm3mc0000gn/T/ipykernel_59394/72639139.py", line 27, in <lambda>
    spark.createDataFrame(preprocessing(rdd)) if not rdd.isEmpty() else None)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/sql/session.py", line 658, in createDataFrame
    raise TypeError("data is already a DataFrame")
TypeError: data is already a DataFrame

	at org.apache.spark.streaming.api.python.TransformFunction.callPythonTransformFunction(PythonDStream.scala:95)
	at org.ap

23/02/26 18:46:04 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:46:04 WARN BlockManager: Block input-0-1677465964600 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:46:05 ERROR JobScheduler: Error running job streaming job 1677465965000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/var/folders/fc/dcskpc694gg68q4bvnmpm3mc0000gn/T/ipykernel_59394/72639139.py", line 27, in <lambda>
    spark.createDataFrame(preprocessing(rdd)) if not rdd.isEmpty() else None)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/sql/session.py", line 658, in createDataFrame
    ra

23/02/26 18:46:08 ERROR JobScheduler: Error running job streaming job 1677465968000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/var/folders/fc/dcskpc694gg68q4bvnmpm3mc0000gn/T/ipykernel_59394/72639139.py", line 27, in <lambda>
    spark.createDataFrame(preprocessing(rdd)) if not rdd.isEmpty() else None)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/sql/session.py", line 658, in createDataFrame
    raise TypeError("data is already a DataFrame")
TypeError: data is already a DataFrame

	at org.apache.spark.streaming.api.python.TransformFunction.callPythonTransformFunction(PythonDStream.scala:95)
	at org.ap

In [5]:
new_count = 0
while new_count < 10:
    
    time.sleep( 3 )
    top_10_tweets = sqlContext.sql( '"SELECT word FROM tweets"' )
    top_10_df = top_10_tweets.toPandas()
    new_count += 1
print(top_10_df)

23/02/26 18:46:08 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:46:08 WARN BlockManager: Block input-0-1677465968600 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:46:09 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:46:09 WARN BlockManager: Block input-0-1677465968800 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:46:09 ERROR JobScheduler: Error running job streaming job 1677465969000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/var/folders/fc/dcskpc694gg68q4bvnmpm3mc0000gn/T/ipykernel_59394/72639139.py", line 27, in <la

23/02/26 18:46:11 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:46:11 WARN BlockManager: Block input-0-1677465971600 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:46:12 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:46:12 WARN BlockManager: Block input-0-1677465971800 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:46:12 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:46:12 WARN BlockManager: Block input-0-1677465972000 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:46:12 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:46:12 WARN BlockManager: Block input-0-1677465972600 replicated to only 0 peer(s) instead of 1 peers


ParseException: 
mismatched input '"SELECT word FROM tweets"' expecting {'(', 'ADD', 'ALTER', 'ANALYZE', 'CACHE', 'CLEAR', 'COMMENT', 'COMMIT', 'CREATE', 'DELETE', 'DESC', 'DESCRIBE', 'DFS', 'DROP', 'EXPLAIN', 'EXPORT', 'FROM', 'GRANT', 'IMPORT', 'INSERT', 'LIST', 'LOAD', 'LOCK', 'MAP', 'MERGE', 'MSCK', 'REDUCE', 'REFRESH', 'REPLACE', 'RESET', 'REVOKE', 'ROLLBACK', 'SELECT', 'SET', 'SHOW', 'START', 'TABLE', 'TRUNCATE', 'UNCACHE', 'UNLOCK', 'UPDATE', 'USE', 'VALUES', 'WITH'}(line 1, pos 0)

== SQL ==
"SELECT word FROM tweets"
^^^


23/02/26 18:46:13 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:46:13 WARN BlockManager: Block input-0-1677465972800 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:46:13 ERROR JobScheduler: Error running job streaming job 1677465972000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/var/folders/fc/dcskpc694gg68q4bvnmpm3mc0000gn/T/ipykernel_59394/72639139.py", line 27, in <lambda>
    spark.createDataFrame(preprocessing(rdd)) if not rdd.isEmpty() else None)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/sql/session.py", line 658, in createDataFrame
    ra

23/02/26 18:46:15 ERROR JobScheduler: Error running job streaming job 1677465975000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/var/folders/fc/dcskpc694gg68q4bvnmpm3mc0000gn/T/ipykernel_59394/72639139.py", line 27, in <lambda>
    spark.createDataFrame(preprocessing(rdd)) if not rdd.isEmpty() else None)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/sql/session.py", line 658, in createDataFrame
    raise TypeError("data is already a DataFrame")
TypeError: data is already a DataFrame

	at org.apache.spark.streaming.api.python.TransformFunction.callPythonTransformFunction(PythonDStream.scala:95)
	at org.ap

23/02/26 18:46:19 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:46:19 WARN BlockManager: Block input-0-1677465979600 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:46:20 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:46:20 WARN BlockManager: Block input-0-1677465979800 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:46:20 ERROR JobScheduler: Error running job streaming job 1677465980000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/var/folders/fc/dcskpc694gg68q4bvnmpm3mc0000gn/T/ipykernel_59394/72639139.py", line 27, in <la

23/02/26 18:46:23 ERROR JobScheduler: Error running job streaming job 1677465983000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/var/folders/fc/dcskpc694gg68q4bvnmpm3mc0000gn/T/ipykernel_59394/72639139.py", line 27, in <lambda>
    spark.createDataFrame(preprocessing(rdd)) if not rdd.isEmpty() else None)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/sql/session.py", line 658, in createDataFrame
    raise TypeError("data is already a DataFrame")
TypeError: data is already a DataFrame

	at org.apache.spark.streaming.api.python.TransformFunction.callPythonTransformFunction(PythonDStream.scala:95)
	at org.ap

23/02/26 18:46:26 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:46:26 WARN BlockManager: Block input-0-1677465986600 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:46:27 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/02/26 18:46:27 WARN BlockManager: Block input-0-1677465986800 replicated to only 0 peer(s) instead of 1 peers
23/02/26 18:46:27 ERROR JobScheduler: Error running job streaming job 1677465987000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/var/folders/fc/dcskpc694gg68q4bvnmpm3mc0000gn/T/ipykernel_59394/72639139.py", line 27, in <la

23/02/26 18:46:30 ERROR JobScheduler: Error running job streaming job 1677465990000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/var/folders/fc/dcskpc694gg68q4bvnmpm3mc0000gn/T/ipykernel_59394/72639139.py", line 27, in <lambda>
    spark.createDataFrame(preprocessing(rdd)) if not rdd.isEmpty() else None)
  File "/Users/Kaustubh/miniconda3/lib/python3.8/site-packages/pyspark/sql/session.py", line 658, in createDataFrame
    raise TypeError("data is already a DataFrame")
TypeError: data is already a DataFrame

	at org.apache.spark.streaming.api.python.TransformFunction.callPythonTransformFunction(PythonDStream.scala:95)
	at org.ap

In [6]:
# stop the SparkSession
ssc.stop()

23/02/26 18:46:32 WARN SocketReceiver: Error receiving data
java.net.SocketException: Socket closed
	at java.net.SocketInputStream.socketRead0(Native Method)
	at java.net.SocketInputStream.socketRead(SocketInputStream.java:116)
	at java.net.SocketInputStream.read(SocketInputStream.java:171)
	at java.net.SocketInputStream.read(SocketInputStream.java:141)
	at sun.nio.cs.StreamDecoder.readBytes(StreamDecoder.java:284)
	at sun.nio.cs.StreamDecoder.implRead(StreamDecoder.java:326)
	at sun.nio.cs.StreamDecoder.read(StreamDecoder.java:178)
	at java.io.InputStreamReader.read(InputStreamReader.java:184)
	at java.io.BufferedReader.fill(BufferedReader.java:161)
	at java.io.BufferedReader.readLine(BufferedReader.java:324)
	at java.io.BufferedReader.readLine(BufferedReader.java:389)
	at org.apache.spark.streaming.dstream.SocketReceiver$$anon$2.getNext(SocketInputDStream.scala:121)
	at org.apache.spark.streaming.dstream.SocketReceiver$$anon$2.getNext(SocketInputDStream.scala:119)
	at org.apache.spar